In [1]:
import pickle
import os

The author accidentally deleted her original notebook by mistake and could not find a way to get it back, so this is an uncomplete version of the notebook used for this experiment.

In [ ]:
# get word embeedings
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
MODEL_NAME = 'jeniya/BERTOverflow'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
model = model.to(DEVICE)

def assign_GPU(Tokenizer_output):
  tokens_tensor = Tokenizer_output['input_ids'].to(DEVICE)
  token_type_ids = Tokenizer_output['token_type_ids'].to(DEVICE)
  attention_mask = Tokenizer_output['attention_mask'].to(DEVICE)
  output = {'input_ids' : tokens_tensor, 
        'token_type_ids' : token_type_ids, 
        'attention_mask' : attention_mask}
  return output

def get_word_embedding(word, sent):
    idx = sent.split(" ").index(word) # get word index
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    input = assign_GPU(encoded)
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    with torch.no_grad():
        output = model(**input)
    states = output.hidden_states
    output = torch.stack([states[i] for i in [-4, -3, -2, -1]]).sum(0).squeeze()
    word_tokens_output = output[token_ids_word]
    word_embedding = word_tokens_output.mean(dim=0)
    return word_embedding

In [ ]:
def get_words_embedding(words, sent):
    sent_words = sent.split(" ")
    idxes = [sent_words.index(w) for w in words] # get word index
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    input = assign_GPU(encoded)
    token_ids_words = [np.where(np.array(encoded.word_ids()) == idx) for idx in idxes]
    with torch.no_grad():
        output = model(**input)
    states = output.hidden_states
    output = torch.stack([states[i] for i in [-4, -3, -2, -1]]).sum(0).squeeze()
    words_tokens_output = [output[token_ids_word] for token_ids_word in token_ids_words]
    word_embeddings = {}
    for i, output in enumerate(words_tokens_output):
      word_embeddings[words[i]] = output.mean(dim=0)
    return word_embeddings

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
# MODEL_NAME = 'jeniya/BERTOverflow'
MODEL_NAME = 'bert-base-uncased'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
model = model.to(DEVICE)

def assign_GPU(Tokenizer_output):
  tokens_tensor = Tokenizer_output['input_ids'].to(DEVICE)
  token_type_ids = Tokenizer_output['token_type_ids'].to(DEVICE)
  attention_mask = Tokenizer_output['attention_mask'].to(DEVICE)
  output = {'input_ids' : tokens_tensor, 
        'token_type_ids' : token_type_ids, 
        'attention_mask' : attention_mask}
  return output

def get_sent_embeddings(sent, encoded):
  input = assign_GPU(encoded)
  with torch.no_grad():
      output = model(**input)
  states = output.hidden_states
  output = torch.stack([states[i] for i in [-4, -3, -2, -1]]).sum(0).squeeze()
  return output

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import os
year = 2020
words = common_words
corpus_file_path = os.path.join("/content/drive/My Drive/CSC2611-Project/data", "all-{}-truncated.txt".format(year))
w2u = {w: [] for w in words} # word -> usages
bunch_size = 10000 
cnt = 0

with open(corpus_file_path, 'r') as f:
    for line in f:
      if len(line) > 512:
          line = line[:512]
      encoded = tokenizer.encode_plus(line, return_tensors="pt")
      sent_embeddings = get_sent_embeddings(line, encoded)
      line_words = line.split(' ')
      for word in words:
        if word in line_words:
          idx = line_words.index(word)
          token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
          word_embedding = sent_embeddings[token_ids_word].mean(dim=0)
          w2u[word].append(word_embedding.to('cpu').numpy())
      cnt += 1
      if cnt % bunch_size == 0: 
        print(cnt, "lines.")
# torch.save(w2u, '/content/drive/My Drive/CSC2611-Project/data/common-words-embeddings-{}-truncated-base.pt'.format(year))

10000 lines.
20000 lines.
30000 lines.


In [ ]:
torch.save(w2u, '/content/drive/My Drive/CSC2611-Project/data/common-words-embeddings-{}-truncated.pt'.format(year))

In [ ]:
# Convert cuda data to cpu
import pickle
year = 2017
with open('/content/drive/My Drive/CSC2611-Project/data/common-words-embeddings-{}-truncated.pkl'.format(year), 'rb') as f:
  cuda_dict = pickle.load(f)
cpu_dict = {}
for w in cuda_dict:
  cpu_dict[w] = [u.to('cpu').numpy for u in cuda_dict[w]]
with open('/content/drive/My Drive/CSC2611-Project/data/common-words-embeddings-{}-truncated-numpy.pt'.format(year), 'wb') as f:
  pickle.dump(cpu_dict, f)

In [ ]:
import numpy as np
from scipy.spatial import distance

def apd(usages1, usages2, metric='cosine'):
    return np.nanmean(distance.cdist(usages1, usages2, metric='cosine'))

In [ ]:
# calculate words with highest apd in 2020
year = 2020
with open('', 'rb') as f:
  word2use_brown = pickle.load(f)
with open(''.format(year), 'rb') as f:
  word2use_so = pickle.load(f)

In [ ]:
sorted_apd = sorted(word2apd, key=(lambda w: -word2apd[w]))
sorted_apd

In [2]:
with open(os.path.join('/data/StackOverflow/Communityposts/data/annual/apd', 'apd-2020-brown-base.pkl'), 'rb') as f:
    apd_2020 = pickle.load(f)

In [6]:
import pandas as pd
df = pd.DataFrame.from_dict(data=apd_2020, orient='index', columns=['APCD'])

In [8]:
df.to_csv('data/apcd-2020.csv')